## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Fortuna,40.5982,-124.1573,57.97,96,90,0.00,US,2021-07-25 17:38:35
1,1,Natchitoches,31.7668,-93.1335,93.20,62,1,0.00,US,2021-07-25 17:38:36
2,2,Port Elizabeth,-33.9180,25.5701,55.69,82,0,1.14,ZA,2021-07-25 17:36:41
3,3,Qaanaaq,77.4840,-69.3632,40.23,70,88,5.48,GL,2021-07-25 17:38:36
4,4,Deputatskiy,69.3000,139.9000,43.09,81,9,3.04,RU,2021-07-25 17:38:37


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 85
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [6]:
preferred_cities_df.notnull().sum()

City_ID       120
City          120
Lat           120
Lng           120
Max Temp      120
Humidity      120
Cloudiness    120
Wind Speed    120
Country       120
Date          120
dtype: int64

In [7]:
preferred_cities_df.count()

City_ID       120
City          120
Lat           120
Lng           120
Max Temp      120
Humidity      120
Cloudiness    120
Wind Speed    120
Country       120
Date          120
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pd.DataFrame(preferred_cities_df)
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Natchitoches,31.7668,-93.1335,93.20,62,1,0.00,US,2021-07-25 17:38:36
6,6,Twentynine Palms,34.1356,-116.0542,92.14,35,81,4.00,US,2021-07-25 17:38:37
8,8,Gazli,40.1333,63.4500,99.63,12,0,15.59,UZ,2021-07-25 17:38:38
10,10,Bemidji,47.4736,-94.8803,87.15,27,1,18.41,US,2021-07-25 17:36:28
11,11,Zhob,31.3411,69.4481,89.26,37,1,6.40,PK,2021-07-25 17:38:38


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
clean_hotel_df = hotel_df

clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Natchitoches,US,93.20,31.7668,-93.1335,
6,Twentynine Palms,US,92.14,34.1356,-116.0542,
8,Gazli,UZ,99.63,40.1333,63.4500,
10,Bemidji,US,87.15,47.4736,-94.8803,
11,Zhob,PK,89.26,31.3411,69.4481,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey

}



# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()





     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the ci

    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError, KeyError):
        print("Hotel not found... skipping.")

    
   
   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.dropna()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Natchitoches,US,93.20,31.7668,-93.1335,Super 8 by Wyndham Natchitoches
6,Twentynine Palms,US,92.14,34.1356,-116.0542,Holiday Inn Express & Suites Twentynine Palms-...
8,Gazli,UZ,99.63,40.1333,63.4500,
10,Bemidji,US,87.15,47.4736,-94.8803,Hampton Inn & Suites Bemidji
11,Zhob,PK,89.26,31.3411,69.4481,ROOM # 105 DUBAI HOTEL ZHOB
...,...,...,...,...,...,...
640,Ellensburg,US,90.25,46.9965,-120.5479,"Holiday Inn Express Ellensburg, an IHG Hotel"
645,Rawah,IQ,99.50,34.4816,41.9093,ديوان الشيخ نامق محسن الراوي
658,Jaleswar,NP,85.21,26.6471,85.8008,Sundar Mobile Repairing Centre
664,Carutapera,BR,87.06,-1.1950,-46.0200,Lidera


In [12]:
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Natchitoches,US,93.20,31.7668,-93.1335,Super 8 by Wyndham Natchitoches
6,Twentynine Palms,US,92.14,34.1356,-116.0542,Holiday Inn Express & Suites Twentynine Palms-...
8,Gazli,UZ,99.63,40.1333,63.4500,
10,Bemidji,US,87.15,47.4736,-94.8803,Hampton Inn & Suites Bemidji
11,Zhob,PK,89.26,31.3411,69.4481,ROOM # 105 DUBAI HOTEL ZHOB


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]






In [15]:
locations


,Lat,Lng
1,31.7668,-93.1335
6,34.1356,-116.0542
8,40.1333,63.4500
10,47.4736,-94.8803
11,31.3411,69.4481
...,...,...
640,46.9965,-120.5479
645,34.4816,41.9093
658,26.6471,85.8008
664,-1.1950,-46.0200


In [16]:
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Natchitoches,US,93.20,31.7668,-93.1335,Super 8 by Wyndham Natchitoches
6,Twentynine Palms,US,92.14,34.1356,-116.0542,Holiday Inn Express & Suites Twentynine Palms-...
8,Gazli,UZ,99.63,40.1333,63.4500,
10,Bemidji,US,87.15,47.4736,-94.8803,Hampton Inn & Suites Bemidji
11,Zhob,PK,89.26,31.3411,69.4481,ROOM # 105 DUBAI HOTEL ZHOB
...,...,...,...,...,...,...
640,Ellensburg,US,90.25,46.9965,-120.5479,"Holiday Inn Express Ellensburg, an IHG Hotel"
645,Rawah,IQ,99.50,34.4816,41.9093,ديوان الشيخ نامق محسن الراوي
658,Jaleswar,NP,85.21,26.6471,85.8008,Sundar Mobile Repairing Centre
664,Carutapera,BR,87.06,-1.1950,-46.0200,Lidera


In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## Deliverable 3. Create a Travel Itinerary Map.

In [18]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df


,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,1,Natchitoches,US,93.20,31.7668,-93.1335,Super 8 by Wyndham Natchitoches
1,6,Twentynine Palms,US,92.14,34.1356,-116.0542,Holiday Inn Express & Suites Twentynine Palms-...
2,8,Gazli,UZ,99.63,40.1333,63.4500,NaN
3,10,Bemidji,US,87.15,47.4736,-94.8803,Hampton Inn & Suites Bemidji
4,11,Zhob,PK,89.26,31.3411,69.4481,ROOM # 105 DUBAI HOTEL ZHOB
...,...,...,...,...,...,...,...
115,640,Ellensburg,US,90.25,46.9965,-120.5479,"Holiday Inn Express Ellensburg, an IHG Hotel"
116,645,Rawah,IQ,99.50,34.4816,41.9093,ديوان الشيخ نامق محسن الراوي
117,658,Jaleswar,NP,85.21,26.6471,85.8008,Sundar Mobile Repairing Centre
118,664,Carutapera,BR,87.06,-1.1950,-46.0200,Lidera


In [19]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 


# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

                               
vacation_start = vacation_df.loc[(vacation_df["City"] == "Vicksburg")] 
vacation_end = vacation_df.loc[(vacation_df["City"] == "Vicksburg")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Simpsonville")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Kinston")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Carroll")]

In [22]:
vacation_start

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
99,563,Vicksburg,US,94.68,32.3526,-90.8779,La Quinta Inn & Suites by Wyndham Vicksburg


In [23]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and 
#list indexing.

start = vacation_start.to_numpy()
end = vacation_end.to_numpy()
stop1 = vacation_stop1.to_numpy()
stop2 = vacation_stop2.to_numpy()
stop3 = vacation_stop3.to_numpy()



stop2


array([[187, 'Kinston', 'US', 88.16, 35.2627, -77.5816,
        'Hampton Inn Kinston']], dtype=object)

In [24]:
stop3[0][1]

'Carroll'

In [25]:
stop1[0][4]

34.7371

In [26]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

# Latitude-longitude pairs
start_tuple = (start[0][4], start[0][5])
end_tuple = (end[0][4], end[0][5])
stop1_tuple = (stop1[0][4], stop1[0][5])
stop2_tuple = (stop2[0][4], stop2[0][5])
stop3_tuple = (stop3[0][4], stop3[0][5])



fig = gmaps.figure()


vicksburg2carroll = gmaps.directions_layer(start_tuple, end_tuple, waypoints = [stop1_tuple, stop2_tuple], travel_mode="BICYCLING")
fig.add_layer(vicksburg2carroll)
fig




Figure(layout=FigureLayout(height='420px'))

In [27]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_end, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df




,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,563,Vicksburg,US,94.68,32.3526,-90.8779,La Quinta Inn & Suites by Wyndham Vicksburg
1,563,Vicksburg,US,94.68,32.3526,-90.8779,La Quinta Inn & Suites by Wyndham Vicksburg
2,621,Simpsonville,US,90.45,34.7371,-82.2543,Hampton Inn Greenville/Simpsonville
3,187,Kinston,US,88.16,35.2627,-77.5816,Hampton Inn Kinston
4,412,Carroll,US,89.78,39.5501,-77.0164,Best Western Westminster Hotel


In [28]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]



locations

,Lat,Lng
0,32.3526,-90.8779
1,32.3526,-90.8779
2,34.7371,-82.2543
3,35.2627,-77.5816
4,39.5501,-77.0164


In [29]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))